<a href="https://colab.research.google.com/github/sibyl-dev/Explingo/blob/basic-llm-api/basic_llm_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
      white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import openai
from google.colab import userdata
import pandas as pd
import numpy as np

In [ ]:
# Perform tasks using OpenAI API
client = openai.OpenAI(
  api_key=userdata.get('openai'),
)

In [ ]:
exp_df = pd.read_csv("ames_housing_0.csv")
exp_df = exp_df.sort_values(by="Contribution", key=abs, ascending=False)
exp = exp_df.to_dict('records')
exp[0]

In [ ]:
def parse_exp(exp, num_features=5):
  features = []
  for i in range(num_features):
    features.append("({}, {}, {})".format(exp[i]['Feature Name'].strip(),
                                          exp[i]['Feature Value'].strip(),
                                          exp[i]['Contribution'],
                                          exp[i]['Average/Mode']))
  return ", ".join(features)

In [ ]:
from openai import OpenAI

prompt =  ("You are a helpful assistant. "
           "You are helping users understand an ML model's predictions. "
           "Do not use more tokens that necessary but make your answers sound natural.")
question =  ("Convert this feature contibution explanation, generated using SHAP, into a simple narrative. "
             "The explanation is presented in (feature, feature_value, contribution, average_feature_value) format: ")
explanation = parse_exp(exp)

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": prompt},
    {"role": "user", "content": question + explanation}
  ],
  max_tokens=128,
  temperature=.5,
  n=5
)

In [ ]:
with open("no-tuning-responses.txt", "w") as f:
  for choice in response.choices:
    f.write(choice.message.content)
    f.write("\n")